In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType

In [0]:
df = spark.table("workspace.bronze.px_cat_g1v2")

In [0]:
df.show()

In [0]:
df.printSchema()

In [0]:
#trim spaces
string_cols = [field.name for field in df.schema.fields if isinstance(field.dataType, StringType)]
for col in string_cols:
    df = df.withColumn(col, F.trim(F.col(col)))

In [0]:
# replace yes/no with boolean values
df = df.withColumn("MAINTENANCE",
       F.when(F.upper(F.col("MAINTENANCE")) == "YES", F.lit(True))
       .when(F.upper(F.col("MAINTENANCE")) == "NO", F.lit(False))
       .otherwise(None)
    )

In [0]:
_RENAME_MAP = {
    "ID": "category_id",
    "CAT": "category",
    "SUBCAT": "subcategory",
    "MAINTENANCE": "maintenance_flag"
}
for old_name, new_name in _RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_cat_g1v2")